In [ ]:
import pandas as pd
import requests
import json
from dotenv import load_dotenv
import os
import time
import numpy as np

In [ ]:
load_dotenv("../../.env",override=True)
consumerKey = os.environ.get("CONSUMER_KEY")


In [ ]:
pd.set_option("display.max_columns",500)
pd.set_option("display.max_rows",20)

In [ ]:
def request_fundamentals(Tickers: list, consumerKey: str) -> pd.DataFrame:
    """_summary_

    Args:
        Tickers (list)
        consumerKey (str)

    Returns:
        pd.DataFrame
    """
    symbolList = ",".join([i for i in Tickers])
    endpoint = f'''https://api.tdameritrade.com/v1/instruments?
        &symbol={symbolList}&projection=fundamental'''
    page = requests.get(url=endpoint, 
        params={'apikey' : consumerKey})
    content = json.loads(page.content)

    allDict = {}
    for k in list(content.keys()):
        allDict[k] = content[k]['fundamental']
    fundamentals = pd.DataFrame(allDict).T
        
    return fundamentals

def request_quotes(consumerKey: str, Tickers: list) -> pd.DataFrame:
    """_summary_

    Args:
        consumerKey (str): _description_
        Tickers (list): _description_

    Returns:
        pd.DataFrame: _description_
    """
    
    symbolList = ",".join([i for i in Tickers])
    endpoint = f"https://api.tdameritrade.com/v1/marketdata/quotes?symbol={symbolList}"
    page = requests.get(url=endpoint, 
                params={'apikey' : consumerKey})
    content = json.loads(page.content)
    content = pd.DataFrame(content).T
    return content

In [ ]:
symbols = pd.read_excel("../../data/static_files/LargeCapFina.xlsx",header=1)
symbols['Symbol'].fillna("-",inplace=True)
Tickers = list(symbols[(~symbols['Symbol'].astype(str).str.contains('-'))]['Symbol'])


In [ ]:

FDF = request_fundamentals(Tickers=Tickers,consumerKey=consumerKey)
Quotes = request_quotes(Tickers=Tickers,consumerKey=consumerKey)

In [ ]:
highGood = ['dividendAmount','dividendYield',
    'grossMarginTTM','grossMarginMRQ','netProfitMarginTTM','netProfitMarginMRQ',
    'operatingMarginTTM','operatingMarginMRQ',
    'returnOnEquity', 'returnOnAssets', 'returnOnInvestment',
    'quickRatio','currentRatio','interestCoverage',
    'epsTTM','epsChangePercentTTM','epsChangeYear','epsChange',
    'revChangeYear', 'revChangeTTM','revChangeIn','marketCap',
    'divGrowthRate3Year','dividendPayAmount','beta']

lowGood = [
    'peRatio', 'pegRatio', 'pbRatio', 'prRatio', 'pcfRatio',
    'totalDebtToCapital', 'ltDebtToEquity', 'totalDebtToEquity',
    'shortIntToFloat', 'shortIntDayToCover'
]

In [ ]:

FDF=pd.DataFrame(np.where(FDF==0,np.nan,FDF),columns = FDF.columns)


In [ ]:
Quotes[['symbol','lastPrice']].merge(FDF[['symbol','peRatio']],
    on='symbol',how='inner')

In [ ]:
FDF